In [1]:
%%capture
!pip install sklearn
!pip install tqdm

In [24]:
import os, sys
sys.path.append('utils/')
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cPickle as pkl
from sklearn.decomposition import PCA
import pandas as pd

from flip_gradient import flip_gradient
from utils import *
from evaluation import compute_ks, roc_auc_truncated
from tqdm import tqdm, trange

SEED = 5

pd.set_option('display.max_columns', 100)

%matplotlib inline

# Preparing data

In [3]:
label_prediction = pd.read_csv('datasets/training.csv.zip').drop('id', 1)
label_prediction = pd.concat([label_prediction, label_prediction]).sample(frac=1., random_state=SEED)
features = label_prediction.drop('signal', 1).columns
print label_prediction.shape
label_prediction.head()

(135106, 50)


,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
21840,0.001173,0.999793,7.309732,0.225475,0.144760,8.930744,4.965856,4945.793457,0.019389,0.015251,0.037411,0.204333,0.249398,1,5,3,1.0,2.0,2.0,5.0,1.000000,0.643805,0.270755,-0.423511,-0.131208,-0.115879,-0.176424,1.225634,1.368773,1.647525,0.339441,0.228714,0.454121,10.899674,11.228765,24.274082,885.767273,1630.131714,2696.752686,8884.096680,12979.206055,17545.503906,2.996217,2.763868,2.559938,572,-99,0,1885.391968,0.464575
56053,0.001524,0.999998,12.253985,0.171148,0.026300,2.110847,7.534899,6609.697754,0.041670,0.008835,0.026718,0.257637,0.093278,0,2,1,0.0,0.0,0.0,0.0,0.867473,0.823890,0.823890,-0.697811,-0.311761,-0.177802,-0.208248,1.368423,1.121142,0.734510,0.154609,0.756436,0.364036,9.368566,44.252838,21.371540,1926.730835,2336.613525,2403.396729,15412.594727,12097.913086,20351.759766,2.768578,2.328018,2.825927,205,1,1,1783.964111,0.572177
45478,0.001228,1.000000,52.646694,1.009517,0.008819,0.769089,8.150103,8629.933594,0.122356,0.008543,0.038060,0.286731,0.024846,0,2,0,0.0,0.0,0.0,0.0,0.934495,0.934495,0.719623,-1.020152,-0.269676,-0.300122,-0.450355,0.691030,1.401399,0.964500,0.575982,0.359241,0.336460,8.886332,30.627249,21.443039,402.388214,4380.829102,3906.171631,10589.078125,108756.500000,133513.781250,3.962947,3.904615,4.224580,300,1,1,1769.629395,0.329092
2583,0.000641,0.999931,5.975876,0.298114,0.069880,3.800114,7.624316,3507.376465,0.158135,0.006316,0.158239,0.096123,0.072929,1,2,1,0.0,1.0,0.0,1.0,0.905173,0.562683,0.465120,-0.250856,-0.186523,-0.036454,-0.027879,1.146612,2.169724,1.156981,0.220572,0.475800,0.106540,3.659899,14.310280,4.953084,386.018738,945.195740,2178.675293,4852.797363,6461.723633,40627.386719,3.222986,2.610014,3.618153,186,-99,0,1669.694946,0.206140
29111,0.000361,0.999051,2.052216,0.136910,0.093032,6.564189,1.652357,6301.269531,0.015981,0.014536,0.004173,0.157119,0.071218,1,2,0,1.0,2.0,0.0,3.0,1.000000,0.617369,0.617369,-0.389445,-0.019306,-0.089797,-0.280342,1.098584,0.679145,1.202928,0.164807,0.121844,0.143390,9.387158,7.297283,8.222139,2004.452393,2760.527100,1727.466309,11180.704102,12460.376953,10097.848633,2.403832,2.187777,2.451416,37,4,1,1773.920410,0.760365


In [4]:
domain_adaptation = pd.read_csv('datasets/domain_adaptation.csv').sample(frac=1., random_state=SEED)
domain_adaptation['mass'] = label_prediction['mass'].mean()
domain_adaptation['min_ANNmuon'] = label_prediction['min_ANNmuon'].mean()
domain_adaptation['production'] = label_prediction['production'].mean()
print domain_adaptation.shape
domain_adaptation.head()

(135106, 50)


,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,domain,mass,min_ANNmuon,production
21840,0.001173,0.999793,7.309732,0.225475,0.144760,8.930744,4.965856,4945.793457,0.019389,0.015251,0.037411,0.204333,0.249398,1,5,3,1.0,2.0,2.0,5.0,1.000000,0.643805,0.270755,-0.423511,-0.131208,-0.115879,-0.176424,1.225634,1.368773,1.647525,0.339441,0.228714,0.454121,10.899674,11.228765,24.274082,885.767273,1630.131714,2696.752686,8884.096680,12979.206055,17545.503906,2.996217,2.763868,2.559938,572,0,1779.389403,0.488508,-36.657765
123606,0.000629,0.999641,2.434156,0.206954,0.070947,4.040024,12.275422,2025.399536,0.078382,0.110501,0.008384,0.096569,0.196643,5,4,1,0.0,0.0,0.0,0.0,0.534862,0.534862,0.431572,-0.965900,-0.339497,-0.239028,-0.387375,2.800599,1.121168,1.042478,0.102599,0.091872,0.378335,5.062073,2.942883,10.952242,1346.521606,573.289978,899.214539,11014.367188,6693.914551,8467.949219,2.791065,3.148871,2.932838,174,1,1779.389403,0.488508,-36.657765
45478,0.001228,1.000000,52.646694,1.009517,0.008819,0.769089,8.150103,8629.933594,0.122356,0.008543,0.038060,0.286731,0.024846,0,2,0,0.0,0.0,0.0,0.0,0.934495,0.934495,0.719623,-1.020152,-0.269676,-0.300122,-0.450355,0.691030,1.401399,0.964500,0.575982,0.359241,0.336460,8.886332,30.627249,21.443039,402.388214,4380.829102,3906.171631,10589.078125,108756.500000,133513.781250,3.962947,3.904615,4.224580,300,0,1779.389403,0.488508,-36.657765
70136,0.002494,0.999941,16.526699,0.304339,0.169334,10.098843,5.769628,4072.138184,0.129421,0.062934,0.024000,0.231940,0.549097,1,0,7,0.0,0.0,0.0,0.0,0.653056,0.452612,0.452612,-1.146727,-0.366491,-0.444720,-0.335516,1.262684,1.626763,0.982080,0.526171,1.607936,0.504395,16.896048,33.279392,29.849096,1226.193848,626.245667,2541.443604,16957.341797,4721.416504,22298.402344,3.318623,2.708841,2.861666,213,1,1779.389403,0.488508,-36.657765
96664,0.002490,0.999978,13.390599,0.425467,0.084934,3.600440,7.263951,906.658447,0.021192,0.105585,0.053239,0.246357,0.455187,2,4,4,1.0,1.0,0.0,2.0,0.370659,0.196871,0.196871,-0.518547,-0.155172,-0.177006,-0.186370,1.183687,0.889904,1.165948,0.868177,0.446425,0.815714,5.662319,14.017855,25.105112,616.326660,584.530273,927.724487,11277.823242,10055.753906,14027.491211,3.599216,3.537393,3.408092,406,1,1779.389403,0.488508,-36.657765


In [5]:
check_agreement = pd.read_csv('datasets/check_agreement.csv.zip')
check_agreement['mass'] = label_prediction['mass'].mean()
check_agreement['min_ANNmuon'] = label_prediction['min_ANNmuon'].mean()
check_agreement['production'] = label_prediction['production'].mean()

print check_agreement.shape
check_agreement.head()

(331147, 52)


,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal,weight,mass,min_ANNmuon,production
0,15347063,0.001451,0.999964,6.945030,0.229196,0.058117,2.961298,7.953543,2251.611816,0.082219,0.084005,0.066887,0.185107,0.214719,8,6,1,2.0,1.0,1.0,4.0,0.732076,0.492269,0.179091,-0.207475,-0.019306,-0.089797,-0.098372,0.606178,0.862549,1.487057,0.483199,0.474925,0.426797,24.701061,10.732132,8.853514,1438.064697,468.645721,834.562378,10392.814453,6380.673828,15195.594727,2.666142,3.302978,3.594246,512,0,-0.307813,1779.389403,0.488508,-36.657765
1,14383299,0.000679,0.999818,9.468235,0.517488,0.189683,14.413060,7.141451,10594.470703,0.007983,0.044154,0.001321,0.039357,0.217507,5,6,17,1.0,1.0,1.0,3.0,0.802508,0.605835,0.584701,-0.659644,-0.278330,-0.186370,-0.194944,1.900118,1.073474,1.336784,0.712242,0.260311,0.123877,11.312134,16.435398,7.737038,316.791351,7547.703613,2861.309814,3174.356934,64480.023438,23134.953125,2.995265,2.834816,2.779366,552,0,-0.331421,1779.389403,0.488508,-36.657765
2,7382797,0.003027,0.999847,13.280714,0.219291,0.231709,11.973175,4.778880,2502.196289,0.045085,0.106614,0.005850,0.335788,0.885080,2,2,1,0.0,0.0,1.0,1.0,0.682607,0.682607,0.295038,-0.399239,-0.115879,-0.131069,-0.152291,0.660675,1.683084,0.798658,0.381544,1.163556,1.290409,16.435801,20.686119,44.521961,1887.477905,317.579529,932.128235,15219.761719,3921.181641,10180.791016,2.776633,3.204923,3.081832,318,0,-0.382215,1779.389403,0.488508,-36.657765
3,6751065,0.000810,0.999998,5.166821,0.167886,0.011298,0.891142,5.528002,5097.813965,0.055115,0.038642,0.003864,0.076522,0.068347,4,4,3,0.0,0.0,0.0,0.0,0.533615,0.533615,0.533615,-0.821041,-0.208248,-0.177802,-0.434991,0.770563,1.093031,0.938619,0.564650,0.164411,0.166646,24.878387,7.873435,9.630725,975.041687,1650.837524,2617.248291,4365.082520,13221.149414,24291.875000,2.179345,2.769762,2.918251,290,0,1.465194,1779.389403,0.488508,-36.657765
4,9439580,0.000706,0.999896,10.897236,0.284975,0.160511,16.367550,8.670339,20388.097656,0.015587,0.020872,0.014612,0.249906,0.139937,0,1,0,0.0,0.0,0.0,0.0,0.926410,0.926410,0.926410,-1.116815,-0.328938,-0.443564,-0.344313,1.080559,1.471946,1.123868,0.373736,0.230584,0.112430,28.557213,18.738485,7.389726,6035.000977,9657.492188,4763.682617,27463.011719,46903.394531,24241.628906,2.196114,2.262732,2.310401,45,0,-0.477084,1779.389403,0.488508,-36.657765


In [7]:
Xd, yd = domain_adaptation[features].values, domain_adaptation.domain.values
Xt, yt = label_prediction[features].values, label_prediction.signal.values

Xd.shape, yd.shape, Xt.shape, yt.shape

((135106, 49), (135106,), (135106, 49), (135106,))

In [8]:
batch_size = 50
features_size = len(features)
features_layer_1 = 100
domain_layer_1 = 20

def build_model(shallow_domain_classifier):
    """
    :param shallow_domain_classifier - if True then shallow domain adaptation will be used
    """
    X = tf.placeholder(tf.float32, [None, features_size], name='X') # Input data
    train = tf.placeholder(tf.bool, [], name='train')       # Switch for routing data to class predictor
    l = tf.placeholder(tf.float32, [], name='l')        # Gradient reversal scaler

    # Feature extractor - single layer
    W0 = weight_variable([features_size, features_layer_1])
    b0 = bias_variable([features_layer_1])
    F = tf.nn.relu(tf.matmul(X, W0) + b0, name='feature')

    # Label predictor - single layer
    Y_ind = tf.placeholder(tf.int32, [None], name='Y_ind')  # Class index
    Y = tf.one_hot(Y_ind, 2)
        
    f = tf.cond(train, lambda: tf.slice(F, [0, 0], [batch_size, -1]), lambda: F)
    y = tf.cond(train, lambda: tf.slice(Y, [0, 0], [batch_size, -1]), lambda: Y)
            
    W1 = weight_variable([features_layer_1, 2])
    b1 = bias_variable([2])
    p_logit = tf.matmul(f, W1) + b1
    p = tf.nn.softmax(p_logit, name='p')
    p_loss = tf.nn.softmax_cross_entropy_with_logits(p_logit, y)
    
    # Optimization
    pred_loss = tf.reduce_sum(p_loss, name='pred_loss')
    pred_train_op = tf.train.AdamOptimizer(0.01).minimize(pred_loss, name='pred_train_op')
    
    # Evaluation
    p_acc = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(y, 1), tf.arg_max(p, 1)), tf.float32), name='p_acc')
    
    # Domain predictor - shallow
    D_ind = tf.placeholder(tf.int32, [None], name='D_ind')  # Domain index
    D = tf.one_hot(D_ind, 2)

    f_ = flip_gradient(F, l)

    if shallow_domain_classifier:
        W2 = weight_variable([features_layer_1, 2])
        b2 = bias_variable([2])
        d_logit = tf.matmul(f_, W2) + b2
        d = tf.nn.softmax(d_logit)
        d_loss = tf.nn.softmax_cross_entropy_with_logits(d_logit, D)

    else:
        W2 = weight_variable([features_layer_1, domain_layer_1])
        b2 = bias_variable([domain_layer_1])
        h2 = tf.nn.relu(tf.matmul(f_, W2) + b2)

        W3 = weight_variable([domain_layer_1, 2])
        b3 = bias_variable([2])
        d_logit = tf.matmul(h2, W3) + b3
        d = tf.nn.softmax(d_logit)
        d_loss = tf.nn.softmax_cross_entropy_with_logits(d_logit, D)
            
    # Optimization
    domain_loss = tf.reduce_sum(d_loss, name='domain_loss')
    domain_train_op = tf.train.AdamOptimizer(0.01).minimize(domain_loss, name='domain_train_op')
        
    # Evaluation
    d_acc = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(D, 1), tf.arg_max(d, 1)), tf.float32), name='d_acc')

In [25]:
sess = tf.InteractiveSession()

def train_and_evaluate(sess, domain_adaptation=None, grad_scale=None, num_batches=10000, verbose=True):
    '''
    :param domain_adaptation - takes None, False or True value. 
    No domain adaptation if None, shallow domain adaptation if False and deep domain adaptation if True
    '''
    sess.close()
    tf.reset_default_graph()
    shallow = False
    if domain_adaptation != None:
        shallow = domain_adaptation
    build_model(shallow)
    sess = tf.InteractiveSession()

    # Create batch builders
    D_batches = batch_generator([Xd, yd], batch_size)
    T_batches = batch_generator([Xt, yt], batch_size)
    
    # Get output tensors and train op
    d_acc = sess.graph.get_tensor_by_name('d_acc:0')
    p_probs = sess.graph.get_tensor_by_name('p:0')
    p_acc = sess.graph.get_tensor_by_name('p_acc:0')
    domain_loss = sess.graph.get_tensor_by_name('domain_loss:0')
    domain_op = sess.graph.get_operation_by_name('domain_train_op')
    target_loss = sess.graph.get_tensor_by_name('pred_loss:0')
    target_op = sess.graph.get_operation_by_name('pred_train_op')
    
    sess.run(tf.initialize_all_variables())
    for i in range(num_batches):
        # If no grad_scale, use a schedule
        if grad_scale is None:
            p = float(i) / num_batches
            lp = 2. / (1. + np.exp(-10. * p)) - 1
        else:
            lp = grad_scale

        X0, y0 = D_batches.next()
        X1, y1 = T_batches.next()
        _, target_loss_value, pa, da = sess.run([target_op, target_loss, p_acc, d_acc],
                                   feed_dict={'X:0': X1, 'Y_ind:0': y1, 'D_ind:0': y0,
                                              'train:0': True, 'l:0': lp})
        if domain_adaptation:
            _, domain_loss_value, pa, da = sess.run([domain_op, domain_loss, p_acc, d_acc],
                                       feed_dict={'X:0': X0, 'Y_ind:0': y1, 'D_ind:0': y0,
                                                  'train:0': True, 'l:0': lp})
        

        if verbose and i % 200 == 0:
            print 'loss: %f, domain accuracy: %f, class accuracy: %f' % (target_loss_value, da, pa)
            if domain_adaptation:
                print 'loss: %f, domain accuracy: %f, class accuracy: %f' % (domain_loss_value, da, pa)
            

            
    # Get final accuracies on whole dataset
    dad, pad = sess.run([d_acc, p_acc], feed_dict={'X:0': Xd, 'Y_ind:0': yd, 
                            'D_ind:0': np.zeros(Xd.shape[0], dtype=np.int32), 'train:0': False, 'l:0': 1.0})
    dat, pat, pt = sess.run([d_acc, p_acc, p_probs], feed_dict={'X:0': Xt, 'Y_ind:0': yt,
                            'D_ind:0': np.ones(Xt.shape[0], dtype=np.int32), 'train:0': False, 'l:0': 1.0})
    agreement_probs = sess.run([p_probs], feed_dict={'X:0': check_agreement[features].values,
                                                     'Y_ind:0': np.zeros_like(check_agreement.signal, dtype=np.int32),
                                                     'D_ind:0': np.ones(check_agreement.shape[0], dtype=np.int32), 
                                                     'train:0': False, 'l:0': 1.0})[0][:, 1]
    
    ks = compute_ks(
        agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    
    auc_trunc = roc_auc_truncated(yt, pt[:, 1])
    print 'Domain accuracy for domain adaptation sample: ', dad
    print 'Target accuracy for domain adaptation sample: ', pad
    print 'Domain accuracy for signal prediction sample: ', dat
    print 'Target accuracy for signal prediction sample: ', pat
    print 'KS: ', ks
    print 'AUC(trunc.): ', auc_trunc
    return pt

def extract_and_plot_pca_feats(sess, feat_tensor_name='feature'):
    F = sess.graph.get_tensor_by_name(feat_tensor_name + ':0')
    emb_s = sess.run(F, feed_dict={'X:0': Xd})
    emb_t = sess.run(F, feed_dict={'X:0': Xt})
    emb_all = np.vstack([emb_s, emb_t])

    pca = PCA(n_components=2)
    pca_emb = pca.fit_transform(emb_all)

    num = pca_emb.shape[0] / 2
    plt.scatter(pca_emb[:num,0], pca_emb[:num,1], c=ys, alpha=0.4)
    plt.scatter(pca_emb[num:,0], pca_emb[num:,1], c=yt, cmap='cool', alpha=0.4)

#### Classification with no domain adaptation

In [26]:
probs_no_domain_adaptation = train_and_evaluate(sess, domain_adaptation=None, verbose=True, num_batches=3000)

loss: 15635.164062, domain accuracy: 0.420000, class accuracy: 0.480000
loss: 1469.939453, domain accuracy: 0.500000, class accuracy: 0.560000
loss: 36.563343, domain accuracy: 0.560000, class accuracy: 0.960000
loss: 182.443115, domain accuracy: 0.540000, class accuracy: 0.920000
loss: 359.417114, domain accuracy: 0.560000, class accuracy: 0.940000
loss: 152.861298, domain accuracy: 0.500000, class accuracy: 0.960000
loss: 9.827987, domain accuracy: 0.460000, class accuracy: 0.980000
loss: 0.000236, domain accuracy: 0.420000, class accuracy: 1.000000
loss: 1.140811, domain accuracy: 0.400000, class accuracy: 0.980000
loss: 67.801018, domain accuracy: 0.420000, class accuracy: 0.940000
loss: 2.263603, domain accuracy: 0.580000, class accuracy: 0.980000
loss: 1.161034, domain accuracy: 0.440000, class accuracy: 1.000000
loss: 7.710919, domain accuracy: 0.400000, class accuracy: 0.980000
loss: 11.827107, domain accuracy: 0.540000, class accuracy: 0.900000
loss: 104.498466, domain accurac

#### Classification with shallow domain adaptation

In [27]:
probs_shallow = train_and_evaluate(sess, domain_adaptation=False, verbose=True, num_batches=3000)

loss: 215217.890625, domain accuracy: 0.500000, class accuracy: 0.620000
loss: 767.221558, domain accuracy: 0.440000, class accuracy: 0.760000
loss: 162.898254, domain accuracy: 0.540000, class accuracy: 0.860000
loss: 205.164642, domain accuracy: 0.440000, class accuracy: 0.840000
loss: 141.493774, domain accuracy: 0.480000, class accuracy: 0.940000
loss: 0.017500, domain accuracy: 0.520000, class accuracy: 1.000000
loss: 1.764858, domain accuracy: 0.440000, class accuracy: 0.980000
loss: 1.484203, domain accuracy: 0.500000, class accuracy: 0.960000
loss: 11.120022, domain accuracy: 0.460000, class accuracy: 0.860000
loss: 0.001137, domain accuracy: 0.580000, class accuracy: 1.000000
loss: 0.031786, domain accuracy: 0.560000, class accuracy: 1.000000
loss: 7.111996, domain accuracy: 0.540000, class accuracy: 0.980000
loss: 0.007096, domain accuracy: 0.480000, class accuracy: 1.000000
loss: 0.004954, domain accuracy: 0.560000, class accuracy: 1.000000
loss: 0.003877, domain accuracy: 0

#### Classification with deep domain adaptation

In [30]:
probs_deep = train_and_evaluate(sess, domain_adaptation=True, verbose=True, num_batches=5000)

loss: 212034.750000, domain accuracy: 0.360000, class accuracy: 0.680000
loss: 113811.328125, domain accuracy: 0.360000, class accuracy: 0.680000
loss: 51338.929688, domain accuracy: 0.520000, class accuracy: 0.360000
loss: 82021.742188, domain accuracy: 0.520000, class accuracy: 0.360000
loss: 5729.188477, domain accuracy: 0.680000, class accuracy: 0.420000
loss: 20762.193359, domain accuracy: 0.680000, class accuracy: 0.420000
loss: 86867.226562, domain accuracy: 0.760000, class accuracy: 0.520000
loss: 31878.863281, domain accuracy: 0.760000, class accuracy: 0.520000
loss: 26913.955078, domain accuracy: 0.640000, class accuracy: 0.560000
loss: 43559.746094, domain accuracy: 0.640000, class accuracy: 0.560000
loss: 10456.988281, domain accuracy: 0.480000, class accuracy: 0.340000
loss: 213117.031250, domain accuracy: 0.480000, class accuracy: 0.340000
loss: 84658.164062, domain accuracy: 0.580000, class accuracy: 0.420000
loss: 85189.804688, domain accuracy: 0.580000, class accuracy: